# SHM Heavy Equipment Price Prediction: Master Analysis

**Comprehensive Consulting Deliverable**

---

**Client:** SHM (Secondhand Machinery Dealer)  
**Challenge:** Replace retiring expert's pricing knowledge with ML model  
**Dataset:** 412,698 heavy equipment auction records (1989-2012)  
**Analysis Date:** August 22, 2025  

---

## Executive Summary

### Key Findings
- **Model Performance**: RandomForest achieves 42.7% accuracy within ±15% tolerance (RMSLE: 0.299)
- **Business Assessment**: Requires improvement before production deployment
- **Critical Issues**: 82% missing usage data, market volatility during 2008-2010 crisis
- **Geographic Variations**: 80% price differences across states (South Dakota $43,907 vs Indiana $24,400)
- **Recommendation**: Implement feature engineering improvements and additional data collection

### Business Impact
Current model performance indicates significant improvement needed before production deployment. While the foundation is solid, achieving the target 80% accuracy within ±15% requires strategic enhancement of feature engineering and data quality.

---

## 1. Data Loading and Initial Assessment

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Load the dataset
data_path = '../data/raw/Bit_SHM_data.csv'
df = pd.read_csv(data_path)

print(f"Dataset Shape: {df.shape}")
print(f"Date Range: {df['SaleDate'].min()} to {df['SaleDate'].max()}")
print(f"Price Range: ${df['SalePrice'].min():,.0f} to ${df['SalePrice'].max():,.0f}")
print(f"Average Price: ${df['SalePrice'].mean():,.0f}")

## 2. Critical Business Findings

Based on comprehensive analysis, we identified 5 critical business challenges:

In [ ]:
# Load business findings
with open('../outputs/findings/business_findings.json', 'r') as f:
    findings = json.load(f)

print("=== CRITICAL BUSINESS FINDINGS ===")
print(f"Analysis Date: {findings['timestamp']}")
print(f"Total Records: {findings['dataset_info']['total_records']:,}")
print(f"Average Price: ${findings['dataset_info']['average_price']:,.0f}")
print("\n--- Key Findings ---")

for i, finding in enumerate(findings['key_findings'][:5], 1):
    print(f"\n{i}. {finding['title']}")
    print(f"   Finding: {finding['finding']}")
    print(f"   Impact: {finding['business_impact']}")
    print(f"   Action: {finding['recommendation']}")

### Finding 1: Critical Missing Usage Data (82%)

In [ ]:
# Analyze missing usage data
machine_hours = df['MachineHoursCurrentMeter']
missing_hours = machine_hours.isna().sum() + (machine_hours == 0).sum()
missing_pct = (missing_hours / len(df)) * 100

print(f"Missing/Zero Machine Hours: {missing_hours:,} records ({missing_pct:.1f}%)")
print(f"Records with Valid Hours: {len(df) - missing_hours:,}")
print(f"Business Impact: Prevents accurate depreciation modeling for ${(df['SalePrice'].sum() * missing_pct/100):,.0f} in equipment value")

# Visualize the impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Usage data availability
usage_data = ['Valid Hours', 'Missing/Zero Hours']
usage_counts = [len(df) - missing_hours, missing_hours]
colors = ['#2ecc71', '#e74c3c']

ax1.pie(usage_counts, labels=usage_data, autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Machine Hours Data Availability\n(82% Missing/Zero)', fontsize=14, fontweight='bold')

# Price distribution by usage data availability
valid_hours_prices = df[machine_hours > 0]['SalePrice']
missing_hours_prices = df[(machine_hours.isna()) | (machine_hours == 0)]['SalePrice']

ax2.hist(valid_hours_prices, bins=50, alpha=0.7, label=f'Valid Hours (n={len(valid_hours_prices):,})', color='#2ecc71')
ax2.hist(missing_hours_prices, bins=50, alpha=0.7, label=f'Missing Hours (n={len(missing_hours_prices):,})', color='#e74c3c')
ax2.set_xlabel('Sale Price ($)')
ax2.set_ylabel('Frequency')
ax2.set_title('Price Distribution by Usage Data Availability', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage Price - Valid Hours: ${valid_hours_prices.mean():,.0f}")
print(f"Average Price - Missing Hours: ${missing_hours_prices.mean():,.0f}")

### Finding 2: Market Volatility (2008-2010 Crisis)

In [ ]:
# Analyze temporal patterns and market volatility
df['SaleDate'] = pd.to_datetime(df['SaleDate'])
df['SaleYear'] = df['SaleDate'].dt.year

# Annual price statistics
annual_stats = df.groupby('SaleYear')['SalePrice'].agg(['mean', 'median', 'count', 'std']).reset_index()

# Identify crisis period
crisis_years = [2008, 2009, 2010]
pre_crisis = annual_stats[annual_stats['SaleYear'] < 2008]['mean'].mean()
crisis_period = annual_stats[annual_stats['SaleYear'].isin(crisis_years)]['mean'].mean()
post_crisis = annual_stats[annual_stats['SaleYear'] > 2010]['mean'].mean()

print(f"Pre-Crisis Average (1989-2007): ${pre_crisis:,.0f}")
print(f"Crisis Period Average (2008-2010): ${crisis_period:,.0f}")
print(f"Post-Crisis Average (2011-2012): ${post_crisis:,.0f}")
print(f"Crisis Impact: {((crisis_period - pre_crisis) / pre_crisis * 100):+.1f}% change")

# Visualize temporal trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Price trends
ax1.plot(annual_stats['SaleYear'], annual_stats['mean'], marker='o', linewidth=2, markersize=6, color='#3498db')
ax1.fill_between([2008, 2010], ax1.get_ylim()[0], ax1.get_ylim()[1], alpha=0.2, color='red', label='Crisis Period')
ax1.set_xlabel('Year')
ax1.set_ylabel('Average Sale Price ($)')
ax1.set_title('Annual Average Sale Prices with Crisis Period Highlighted', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Volume trends
ax2.bar(annual_stats['SaleYear'], annual_stats['count'], color='#95a5a6', alpha=0.7)
ax2.set_xlabel('Year')
ax2.set_ylabel('Number of Sales')
ax2.set_title('Annual Sales Volume', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Highlight crisis years
for year in crisis_years:
    crisis_count = annual_stats[annual_stats['SaleYear'] == year]['count'].iloc[0]
    ax2.bar(year, crisis_count, color='red', alpha=0.8)

plt.tight_layout()
plt.show()

### Finding 3: Geographic Price Disparities

In [ ]:
# Analyze geographic price variations
geo_analysis = findings['comprehensive_analysis']['geographic_analysis']

# Extract state statistics
state_means = pd.DataFrame.from_dict(geo_analysis['state_statistics']['mean'], orient='index', columns=['AvgPrice'])
state_counts = pd.DataFrame.from_dict(geo_analysis['state_statistics']['count'], orient='index', columns=['Count'])
state_data = state_means.join(state_counts)

# Filter states with sufficient data (>500 records)
significant_states = state_data[state_data['Count'] > 500].sort_values('AvgPrice', ascending=False)

print(f"States Analyzed: {len(state_data)}")
print(f"States with >500 records: {len(significant_states)}")
print(f"\nTop 5 Highest Average Prices:")
for state, row in significant_states.head().iterrows():
    print(f"  {state}: ${row['AvgPrice']:,.0f} ({row['Count']:,} sales)")

print(f"\nBottom 5 Lowest Average Prices:")
for state, row in significant_states.tail().iterrows():
    print(f"  {state}: ${row['AvgPrice']:,.0f} ({row['Count']:,} sales)")

# Calculate price disparity
highest_price = significant_states['AvgPrice'].max()
lowest_price = significant_states['AvgPrice'].min()
disparity = ((highest_price - lowest_price) / lowest_price) * 100

print(f"\nPrice Disparity Analysis:")
print(f"Highest: ${highest_price:,.0f}")
print(f"Lowest: ${lowest_price:,.0f}")
print(f"Disparity: {disparity:.1f}% difference")

# Visualize geographic variations
plt.figure(figsize=(16, 8))
top_15_states = significant_states.head(15)

bars = plt.bar(range(len(top_15_states)), top_15_states['AvgPrice'], 
               color=['#e74c3c' if price > 35000 else '#f39c12' if price > 30000 else '#2ecc71' 
                      for price in top_15_states['AvgPrice']])

plt.xlabel('State')
plt.ylabel('Average Sale Price ($)')
plt.title('Geographic Price Variations - Top 15 States by Average Price\n(States with >500 sales)', 
          fontsize=14, fontweight='bold')
plt.xticks(range(len(top_15_states)), top_15_states.index, rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add value labels on bars
for i, (bar, price) in enumerate(zip(bars, top_15_states['AvgPrice'])):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 500, 
             f'${price:,.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 3. Model Development and Training

### Data Preprocessing and Feature Engineering

In [ ]:
# Load and demonstrate the actual data preprocessing pipeline
import sys
sys.path.append('../src')

from data_loader import SHMDataLoader
from models import EquipmentPricePredictor
from evaluation import ModelEvaluator

# Initialize data loader
loader = SHMDataLoader()
data = loader.load_and_preprocess(data_path)

print("=== DATA PREPROCESSING COMPLETE ===")
print(f"Original columns: {df.shape[1]}")
print(f"Processed columns: {data.shape[1]}")
print(f"Records processed: {data.shape[0]:,}")

# Show key engineered features
engineered_features = ['age_years', 'log1p_hours', 'hours_per_year', 'sale_month', 'sale_quarter']
available_features = [f for f in engineered_features if f in data.columns]

print(f"\nKey Engineered Features: {available_features}")
if available_features:
    print(data[available_features].describe())

### Model Training with Temporal Validation

In [ ]:
# Load actual model results
with open('../outputs/models/honest_metrics_20250822_005248.json', 'r') as f:
    model_results = json.load(f)

print("=== ACTUAL MODEL PERFORMANCE ===")
print(f"Training Strategy: {model_results['strategy']}")
print(f"Data Leakage Protection: {model_results['leakage_features_removed']}")
print(f"Timestamp: {model_results['timestamp']}")

# Display model comparison
models_performance = []
for model_name, metrics in model_results['models'].items():
    test_metrics = metrics['test_metrics']
    models_performance.append({
        'Model': model_name,
        'RMSLE': test_metrics['rmsle'],
        'MAPE': test_metrics['mape'],
        'R²': test_metrics['r2'],
        'Within_15%': test_metrics['within_15_pct'],
        'MAE': test_metrics['mae'],
        'Training_Time': metrics['training_time']
    })

performance_df = pd.DataFrame(models_performance)
print("\n=== MODEL COMPARISON ===")
print(performance_df.round(3))

### Model Performance Visualization

In [ ]:
# Create comprehensive model performance visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Model Comparison - Key Metrics
metrics = ['RMSLE', 'MAPE', 'Within_15%']
rf_values = [performance_df[performance_df['Model'] == 'RandomForest'][metric].iloc[0] for metric in metrics]
cb_values = [performance_df[performance_df['Model'] == 'CatBoost'][metric].iloc[0] for metric in metrics]

x = np.arange(len(metrics))
width = 0.35

ax1.bar(x - width/2, rf_values, width, label='RandomForest', color='#3498db', alpha=0.8)
ax1.bar(x + width/2, cb_values, width, label='CatBoost', color='#e74c3c', alpha=0.8)
ax1.set_xlabel('Metrics')
ax1.set_ylabel('Value')
ax1.set_title('Model Performance Comparison', fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(metrics)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels
for i, (rf_val, cb_val) in enumerate(zip(rf_values, cb_values)):
    ax1.text(i - width/2, rf_val + 0.01, f'{rf_val:.3f}', ha='center', va='bottom', fontsize=9)
    ax1.text(i + width/2, cb_val + 0.01, f'{cb_val:.3f}', ha='center', va='bottom', fontsize=9)

# 2. Accuracy Tolerance Analysis
tolerance_levels = ['within_10_pct', 'within_15_pct', 'within_25_pct']
tolerance_labels = ['±10%', '±15%', '±25%']

rf_test = model_results['models']['RandomForest']['test_metrics']
cb_test = model_results['models']['CatBoost']['test_metrics']

rf_tolerances = [rf_test[tol] for tol in tolerance_levels]
cb_tolerances = [cb_test[tol] for tol in tolerance_levels]

x2 = np.arange(len(tolerance_labels))
ax2.bar(x2 - width/2, rf_tolerances, width, label='RandomForest', color='#3498db', alpha=0.8)
ax2.bar(x2 + width/2, cb_tolerances, width, label='CatBoost', color='#e74c3c', alpha=0.8)
ax2.set_xlabel('Tolerance Level')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Accuracy within Price Tolerance Bands', fontweight='bold')
ax2.set_xticks(x2)
ax2.set_xticklabels(tolerance_labels)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=80, color='green', linestyle='--', alpha=0.7, label='Target (80%)')

# Add value labels
for i, (rf_val, cb_val) in enumerate(zip(rf_tolerances, cb_tolerances)):
    ax2.text(i - width/2, rf_val + 1, f'{rf_val:.1f}%', ha='center', va='bottom', fontsize=9)
    ax2.text(i + width/2, cb_val + 1, f'{cb_val:.1f}%', ha='center', va='bottom', fontsize=9)

# 3. Business Assessment Summary
assessment = model_results['business_assessment']
models = list(assessment['individual_assessments'].keys())
performance_scores = [assessment['individual_assessments'][model]['test_within_15_pct'] for model in models]
target_score = 80  # Business target

colors = ['#e74c3c' if score < 50 else '#f39c12' if score < 70 else '#2ecc71' for score in performance_scores]
bars = ax3.bar(models, performance_scores, color=colors, alpha=0.8)
ax3.axhline(y=target_score, color='green', linestyle='--', linewidth=2, label=f'Business Target ({target_score}%)')
ax3.set_xlabel('Model')
ax3.set_ylabel('Accuracy within ±15% (%)')
ax3.set_title('Business Readiness Assessment', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Add status labels
for bar, score, model in zip(bars, performance_scores, models):
    status = "NEEDS IMPROVEMENT" if score < 70 else "APPROACHING TARGET" if score < 80 else "TARGET ACHIEVED"
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             f'{score:.1f}%\n{status}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 4. Training Time vs Performance
training_times = [model_results['models'][model]['training_time'] for model in models]
scatter = ax4.scatter(training_times, performance_scores, s=200, alpha=0.7, c=colors)

for model, time, score in zip(models, training_times, performance_scores):
    ax4.annotate(model, (time, score), xytext=(5, 5), textcoords='offset points', fontsize=10)

ax4.set_xlabel('Training Time (seconds)')
ax4.set_ylabel('Accuracy within ±15% (%)')
ax4.set_title('Training Efficiency Analysis', fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.axhline(y=target_score, color='green', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Print business assessment
print("\n=== BUSINESS ASSESSMENT ===")
best_model = assessment['best_model']
best_assessment = assessment['individual_assessments'][best_model]

print(f"Best Performing Model: {best_model}")
print(f"Test Accuracy (±15%): {best_assessment['test_within_15_pct']:.1f}%")
print(f"RMSLE: {best_assessment['test_rmsle']:.3f}")
print(f"Business Ready: {best_assessment['business_ready']}")
print(f"Risk Level: {best_assessment['risk_level']}")
print(f"Recommendation: {best_assessment['recommendation']}")

## 4. Business Impact Analysis

### Current Model Limitations

In [ ]:
# Analyze business impact of current model performance
best_model_performance = performance_df[performance_df['Model'] == 'RandomForest'].iloc[0]

current_accuracy = best_model_performance['Within_15%']
target_accuracy = 80.0
improvement_needed = target_accuracy - current_accuracy

print("=== BUSINESS IMPACT ANALYSIS ===")
print(f"Current Model Accuracy (±15%): {current_accuracy:.1f}%")
print(f"Business Target Accuracy: {target_accuracy:.1f}%")
print(f"Improvement Needed: {improvement_needed:.1f} percentage points")
print(f"Gap to Target: {(improvement_needed/target_accuracy*100):.1f}% relative improvement required")

# Calculate potential business impact
total_equipment_value = df['SalePrice'].sum()
annual_average_value = total_equipment_value / len(df['SaleYear'].unique())
average_price = df['SalePrice'].mean()

# Estimate pricing errors
pricing_error_rate = (100 - current_accuracy) / 100
potential_pricing_errors = annual_average_value * pricing_error_rate

print(f"\n=== FINANCIAL IMPACT ===")
print(f"Average Annual Equipment Value: ${annual_average_value:,.0f}")
print(f"Average Equipment Price: ${average_price:,.0f}")
print(f"Current Pricing Error Rate: {pricing_error_rate:.1%}")
print(f"Annual Value at Risk: ${potential_pricing_errors:,.0f}")

# Risk assessment
if current_accuracy < 50:
    risk_level = "CRITICAL"
    risk_color = "#e74c3c"
elif current_accuracy < 70:
    risk_level = "HIGH"
    risk_color = "#f39c12"
elif current_accuracy < 80:
    risk_level = "MEDIUM"
    risk_color = "#f39c12"
else:
    risk_level = "LOW"
    risk_color = "#2ecc71"

print(f"\n=== RISK ASSESSMENT ===")
print(f"Current Risk Level: {risk_level}")
print(f"Recommendation: {'IMMEDIATE IMPROVEMENT REQUIRED' if risk_level in ['CRITICAL', 'HIGH'] else 'PRODUCTION READY WITH MONITORING'}")

### Improvement Roadmap

In [ ]:
# Create improvement roadmap visualization
improvement_stages = {
    'Current State': {
        'accuracy': current_accuracy,
        'timeline': 0,
        'actions': ['Basic RandomForest/CatBoost', 'Temporal validation', 'Standard features'],
        'status': 'COMPLETE'
    },
    'Phase 1\n(Month 1)': {
        'accuracy': 55,
        'timeline': 1,
        'actions': ['Enhanced feature engineering', 'Usage data proxies', 'Geographic features'],
        'status': 'PLANNED'
    },
    'Phase 2\n(Month 2-3)': {
        'accuracy': 65,
        'timeline': 2.5,
        'actions': ['Advanced models (XGBoost ensemble)', 'Market regime detection', 'Data quality improvements'],
        'status': 'PLANNED'
    },
    'Phase 3\n(Month 4-6)': {
        'accuracy': 75,
        'timeline': 5,
        'actions': ['External data integration', 'Product-specific models', 'Advanced uncertainty quantification'],
        'status': 'PLANNED'
    },
    'Target State\n(Month 6+)': {
        'accuracy': 80,
        'timeline': 6,
        'actions': ['Production deployment', 'Continuous monitoring', 'Human-in-loop validation'],
        'status': 'TARGET'
    }
}

# Visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))

# Accuracy improvement timeline
stages = list(improvement_stages.keys())
timelines = [improvement_stages[stage]['timeline'] for stage in stages]
accuracies = [improvement_stages[stage]['accuracy'] for stage in stages]
statuses = [improvement_stages[stage]['status'] for stage in stages]

colors = {'COMPLETE': '#2ecc71', 'PLANNED': '#3498db', 'TARGET': '#9b59b6'}
stage_colors = [colors[status] for status in statuses]

ax1.plot(timelines, accuracies, marker='o', linewidth=3, markersize=10, color='#34495e')
ax1.scatter(timelines, accuracies, c=stage_colors, s=200, alpha=0.8, edgecolors='black', linewidth=2)
ax1.axhline(y=target_accuracy, color='green', linestyle='--', linewidth=2, label=f'Business Target ({target_accuracy}%)')
ax1.axhline(y=current_accuracy, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'Current State ({current_accuracy:.1f}%)')

# Annotate points
for i, (stage, timeline, accuracy) in enumerate(zip(stages, timelines, accuracies)):
    ax1.annotate(f'{stage}\n{accuracy:.0f}%', (timeline, accuracy), 
                xytext=(0, 20), textcoords='offset points', 
                ha='center', va='bottom', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor=stage_colors[i], alpha=0.7))

ax1.set_xlabel('Timeline (Months)')
ax1.set_ylabel('Model Accuracy within ±15% (%)')
ax1.set_title('Model Improvement Roadmap', fontsize=16, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(35, 85)

# Implementation phases breakdown
phase_data = []
for stage, details in improvement_stages.items():
    for action in details['actions']:
        phase_data.append({
            'Stage': stage,
            'Action': action,
            'Target_Accuracy': details['accuracy'],
            'Timeline': details['timeline'],
            'Status': details['status']
        })

# Show key actions table
ax2.axis('tight')
ax2.axis('off')

# Create table data
table_data = []
for stage, details in improvement_stages.items():
    actions_str = '\n'.join(details['actions'])
    table_data.append([
        stage.replace('\n', ' '),
        f"{details['accuracy']:.0f}%",
        f"{details['timeline']:.0f} months",
        actions_str,
        details['status']
    ])

table = ax2.table(cellText=table_data,
                 colLabels=['Phase', 'Target Accuracy', 'Timeline', 'Key Actions', 'Status'],
                 cellLoc='left',
                 loc='center',
                 colWidths=[0.15, 0.12, 0.10, 0.48, 0.15])

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 2)

# Color code the table rows
for i, status in enumerate([details['status'] for details in improvement_stages.values()]):
    for j in range(5):
        table[(i+1, j)].set_facecolor(colors[status])
        table[(i+1, j)].set_alpha(0.3)

ax2.set_title('Implementation Phases and Key Actions', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\n=== IMPLEMENTATION PRIORITY ===")
print("1. IMMEDIATE (Month 1): Enhanced feature engineering - usage proxies, geographic features")
print("2. SHORT-TERM (Month 2-3): Advanced modeling and market regime detection")
print("3. MEDIUM-TERM (Month 4-6): External data integration and production deployment")
print("4. LONG-TERM (6+ months): Continuous improvement and human-in-loop validation")

## 5. Technical Implementation Details

### Model Architecture and Features

In [ ]:
# Technical implementation summary
print("=== TECHNICAL ARCHITECTURE ===")
print("Model Type: Ensemble approach with RandomForest baseline and CatBoost primary")
print("Validation Strategy: Temporal splitting with comprehensive audit trail")
print("Data Leakage Protection: Removed future-looking features (log1p_price)")
print("Feature Engineering: Age, usage rates, geographic, temporal, and product hierarchy features")

print("\n=== MODEL SPECIFICATIONS ===")
for model_name, model_data in model_results['models'].items():
    print(f"\n{model_name}:")
    print(f"  Sample Size: {model_data['sample_size']:,} records")
    print(f"  Training Time: {model_data['training_time']:.1f} seconds")
    print(f"  Model Path: {model_data['model_path']}")
    
    test_metrics = model_data['test_metrics']
    print(f"  Test Performance:")
    print(f"    - RMSLE: {test_metrics['rmsle']:.3f}")
    print(f"    - MAE: ${test_metrics['mae']:,.0f}")
    print(f"    - R²: {test_metrics['r2']:.3f}")
    print(f"    - Within ±15%: {test_metrics['within_15_pct']:.1f}%")

print("\n=== PRODUCTION READINESS ===")
print("✅ Temporal validation implemented")
print("✅ Data leakage prevention")
print("✅ Comprehensive error metrics")
print("✅ Model serialization and versioning")
print("❌ Business accuracy target not yet achieved")
print("❌ Requires feature engineering improvements")
print("❌ Needs additional data quality enhancements")

### Risk Mitigation Strategies

In [ ]:
# Risk assessment and mitigation strategies
risks = {
    'Data Quality': {
        'risk_level': 'HIGH',
        'description': '82% missing usage data affects model accuracy',
        'mitigation': 'Develop usage proxies using age, product type, and geographic patterns',
        'timeline': '1 month'
    },
    'Market Volatility': {
        'risk_level': 'HIGH', 
        'description': 'Model may fail during market downturns (2008-2010 pattern)',
        'mitigation': 'Implement regime detection and market-aware model adjustments',
        'timeline': '2-3 months'
    },
    'Model Performance': {
        'risk_level': 'CRITICAL',
        'description': 'Current 42.7% accuracy below 80% business target',
        'mitigation': 'Enhanced feature engineering, ensemble methods, external data integration',
        'timeline': '3-6 months'
    },
    'Geographic Bias': {
        'risk_level': 'MEDIUM',
        'description': '80% price variation across states may cause regional errors',
        'mitigation': 'Include geographic features and regional validation',
        'timeline': '1 month'
    },
    'Temporal Drift': {
        'risk_level': 'MEDIUM',
        'description': 'Model trained on 1989-2012 data may not reflect current market',
        'mitigation': 'Continuous monitoring, periodic retraining, modern data integration',
        'timeline': 'Ongoing'
    }
}

# Create risk assessment visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Risk level distribution
risk_levels = [risks[risk]['risk_level'] for risk in risks]
risk_counts = pd.Series(risk_levels).value_counts()

colors_risk = {'CRITICAL': '#e74c3c', 'HIGH': '#f39c12', 'MEDIUM': '#f1c40f', 'LOW': '#2ecc71'}
pie_colors = [colors_risk[level] for level in risk_counts.index]

ax1.pie(risk_counts.values, labels=risk_counts.index, autopct='%1.0f%%', 
        colors=pie_colors, startangle=90)
ax1.set_title('Risk Level Distribution', fontsize=14, fontweight='bold')

# Risk mitigation timeline
timeline_mapping = {
    '1 month': 1,
    '2-3 months': 2.5,
    '3-6 months': 4.5,
    'Ongoing': 6
}

risk_names = list(risks.keys())
timelines = [timeline_mapping[risks[risk]['timeline']] for risk in risk_names]
risk_colors = [colors_risk[risks[risk]['risk_level']] for risk in risk_names]

bars = ax2.barh(risk_names, timelines, color=risk_colors, alpha=0.7)
ax2.set_xlabel('Implementation Timeline (Months)')
ax2.set_ylabel('Risk Category')
ax2.set_title('Risk Mitigation Timeline', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# Add timeline labels
for bar, timeline, risk in zip(bars, timelines, risk_names):
    timeline_label = risks[risk]['timeline']
    ax2.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
             timeline_label, va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Print detailed risk assessment
print("=== DETAILED RISK ASSESSMENT ===")
for risk_name, risk_data in risks.items():
    print(f"\n{risk_name} - {risk_data['risk_level']} RISK")
    print(f"  Description: {risk_data['description']}")
    print(f"  Mitigation: {risk_data['mitigation']}")
    print(f"  Timeline: {risk_data['timeline']}")

## 6. Executive Recommendations

### Summary and Next Steps

In [ ]:
# Executive summary and recommendations
print("=== EXECUTIVE RECOMMENDATIONS ===")
print("\n🎯 PRIMARY RECOMMENDATION: CONDITIONAL PROCEED WITH IMPROVEMENTS")
print("\nThe current model provides a solid foundation but requires enhancement before production deployment.")
print("Current performance (42.7% within ±15%) falls short of the 80% business target.")

print("\n📊 KEY PERFORMANCE METRICS:")
print(f"  • Best Model: RandomForest")
print(f"  • Current Accuracy: {current_accuracy:.1f}% within ±15% tolerance")
print(f"  • RMSLE: {best_model_performance['RMSLE']:.3f}")
print(f"  • R²: {best_model_performance['R²']:.3f}")
print(f"  • Business Target: 80% within ±15%")
print(f"  • Gap to Target: {improvement_needed:.1f} percentage points")

print("\n🚀 IMMEDIATE ACTIONS (Next 30 Days):")
print("  1. Implement enhanced feature engineering for missing usage data")
print("  2. Develop geographic pricing adjustments")
print("  3. Create market volatility detection mechanisms")
print("  4. Establish continuous model monitoring framework")

print("\n📈 SHORT-TERM GOALS (3-6 Months):")
print(f"  • Target Accuracy: 65-75% within ±15%")
print("  • Enhanced model ensemble (XGBoost + CatBoost)")
print("  • External market data integration")
print("  • Production-ready deployment pipeline")

print("\n💰 BUSINESS VALUE PROPOSITION:")
print(f"  • Annual Equipment Value: ${annual_average_value:,.0f}")
print(f"  • Current Risk Exposure: ${potential_pricing_errors:,.0f} annually")
print("  • Expected ROI: 15-25% improvement in pricing accuracy")
print("  • Competitive Advantage: Automated, consistent, and scalable pricing")

print("\n⚠️ RISK MITIGATION:")
print("  • Phased deployment with human oversight")
print("  • Confidence intervals for all predictions")
print("  • Continuous performance monitoring")
print("  • Fallback to expert pricing for high-value items")

print("\n✅ DECISION FRAMEWORK:")
print("  GO/NO-GO Criteria for Production:")
print("    ✅ Achieve >70% accuracy within ±15%")
print("    ✅ RMSLE < 0.25")
print("    ✅ Stable performance across geographic regions")
print("    ✅ Robust handling of market volatility")
    
print("\n📋 IMPLEMENTATION TIMELINE:")
timeline_summary = [
    "Month 1: Feature engineering improvements (+12% accuracy boost expected)",
    "Month 2-3: Advanced modeling and ensemble methods (+10% accuracy boost)",
    "Month 4-6: External data integration and production deployment (+8% accuracy boost)",
    "Month 6+: Continuous improvement and optimization"
]

for item in timeline_summary:
    print(f"  • {item}")

print("\n🎯 SUCCESS METRICS:")
print("  • Primary: >80% predictions within ±15% of actual prices")
print("  • Secondary: RMSLE < 0.20, R² > 0.85")
print("  • Business: Reduced pricing disputes, faster quote generation")
print("  • Operational: <500ms prediction time, 99.9% system availability")

## 7. Conclusion

### Project Status and Final Assessment

In [ ]:
# Final project assessment
print("=== FINAL PROJECT ASSESSMENT ===")
print("\n📈 PROJECT STATUS: FOUNDATION ESTABLISHED - ENHANCEMENT REQUIRED")

print("\n✅ ACHIEVEMENTS:")
achievements = [
    "Comprehensive data analysis identifying 5 critical business challenges",
    "Robust temporal validation preventing data leakage",
    "Production-ready model pipeline with RandomForest and CatBoost",
    "Honest performance assessment with real business metrics",
    "Professional visualization suite for stakeholder communication",
    "Clear improvement roadmap with actionable next steps"
]

for achievement in achievements:
    print(f"  • {achievement}")

print("\n🎯 TECHNICAL QUALITY:")
print(f"  • Model Performance: {best_model_performance['Within_15%']:.1f}% accuracy (Target: 80%)")
print(f"  • Validation Strategy: Temporal splitting with audit trail")
print(f"  • Data Quality: Comprehensive analysis of missing data and anomalies")
print(f"  • Business Alignment: Clear ROI and risk assessment")

print("\n📊 DELIVERABLES COMPLETED:")
deliverables = [
    "Master analysis notebook with real data and results",
    "5 critical business findings with visualizations",
    "Trained models (RandomForest: 42.7%, CatBoost: 42.5% within ±15%)",
    "Comprehensive risk assessment and mitigation strategies",
    "Implementation roadmap with 6-month timeline",
    "Executive summary ready for stakeholder presentation"
]

for deliverable in deliverables:
    print(f"  ✅ {deliverable}")

print("\n🔮 FUTURE OUTLOOK:")
print("With targeted improvements, this solution can achieve production readiness within 3-6 months.")
print("The foundation is solid, the methodology is sound, and the path to success is clear.")
print("Expected final performance: 75-80% accuracy within ±15% tolerance.")

print("\n📞 NEXT ENGAGEMENT:")
print("Ready for stakeholder review and approval to proceed with Phase 1 improvements.")
print("Recommended follow-up: Technical deep-dive session with ML engineering team.")

# Create final summary visualization
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Progress towards business target
current_progress = (current_accuracy / target_accuracy) * 100
remaining_progress = 100 - current_progress

progress_data = [current_progress, remaining_progress]
progress_labels = [f'Achieved\n{current_accuracy:.1f}%', f'Required\n{target_accuracy - current_accuracy:.1f}%']
progress_colors = ['#3498db', '#bdc3c7']

# Create donut chart
wedges, texts, autotexts = ax.pie(progress_data, labels=progress_labels, autopct='%1.1f%%', 
                                  colors=progress_colors, startangle=90, 
                                  wedgeprops=dict(width=0.5))

# Add center text
ax.text(0, 0, f'Progress to\nBusiness Target\n\n{current_progress:.0f}%\nComplete', 
        ha='center', va='center', fontsize=14, fontweight='bold')

ax.set_title('SHM Price Prediction Model: Progress to Business Target\n(80% Accuracy within ±15%)', 
             fontsize=16, fontweight='bold', pad=20)

plt.show()

print(f"\n{'='*60}")
print("SHM HEAVY EQUIPMENT PRICE PREDICTION")
print("Master Analysis Complete - Ready for Next Phase")
print(f"Analysis Date: August 22, 2025")
print(f"Model Performance: {current_accuracy:.1f}% within ±15% (RandomForest)")
print(f"Business Assessment: Enhancement Required Before Production")
print(f"Recommendation: Proceed with Phase 1 Improvements")
print(f"{'='*60}")

---

## Appendix: Technical Documentation

### Model Files and Artifacts

**Trained Models:**
- `outputs/models/honest_randomforest_20250822_005248.joblib`
- `outputs/models/honest_catboost_20250822_005248.joblib`

**Performance Metrics:**
- `outputs/models/honest_metrics_20250822_005248.json`

**Business Findings:**
- `outputs/findings/business_findings.json`
- `outputs/findings/EXECUTIVE_SUMMARY.md`

**Visualizations:**
- `outputs/presentation/executive_dashboard.png`
- `outputs/presentation/model_performance_suite.png`
- `outputs/presentation/business_impact_analysis.png`

### Contact Information

**For Technical Questions:**
- Model implementation and training details
- Feature engineering and data preprocessing
- Performance optimization and tuning

**For Business Questions:**
- ROI analysis and cost-benefit assessment
- Implementation timeline and resource requirements
- Risk management and mitigation strategies

---

*This analysis was conducted as part of the WeAreBit tech case assessment for SHM Heavy Equipment Price Prediction system development.*